In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re

import pandas as pd
from gensim.corpora import Dictionary
from nltk import word_tokenize as nltk_word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from tqdm.notebook import tqdm
tqdm.pandas()

from dynamic_bernoulli_embeddings.training import train_model

/Users/lukelefebure/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
def _bad_word(word):
    if len(word) < 2:
        return True
    if any(c.isdigit() for c in word):
        return True
    if "/" in word:
        return True
    return False

def word_tokenize(text):
    text = re.sub(r"-", " ", text)
    text = re.sub("co-operation", "cooperation", text)
    words = [w.lower().strip("'.") for w in nltk_word_tokenize(text)]
    words = [w for w in words if not _bad_word(w)]
    return words

In [5]:
dataset = pd.read_csv("data/un-general-debates.csv")
dataset["bow"] = dataset.text.progress_apply(word_tokenize)
dataset["time"] = dataset.year - dataset.year.min()

In [9]:
# Generate dictionary.
dictionary = Dictionary(dataset.bow)
dictionary.filter_extremes(no_below=15, no_above=1.)
dictionary.compactify()
print(len(dictionary))

4770


In [4]:
model, loss_history, val_history = train_model(
    dataset, dictionary.token2id, validation=.1, num_epochs=2)